In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from itertools import product
from tqdm import tqdm_notebook
#from heuristi_plot_1 import plot as hplot

path = '../output_data/'

In [ ]:
with pd.HDFStore(path + 'X2.h5') as store:
    data = store.select('dat_1', where='n_cp < 21', columns=['decision state', 'i_c', 'G'])
data.index = data.index.droplevel(['test'])
data = data.reset_index('tstep')
data = data.reset_index().drop_duplicates(['phi', 'n_cp', 'sample', 'tstep']).set_index(['phi', 'n_cp', 'sample', 'tstep'])
data.index.levels

G_start = data[['G']].max()[0]

In [ ]:
c1 = 0.43 # tons of co2 per barrel oil https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references
c2 = 7.33 # barrels per toe https://en.wikipedia.org/wiki/Tonne_of_oil_equivalent
c3 = 10e6 # tons per megaton

# carbon budgets from 2011 onwards
b15p66 = 400 # Giga tones of co2 https://www.carbontracker.org/carbon-budgets-explained/ and IPCC2013
b15p50 = 550
b15p33 = 850
b20p66 = 1000

x1 = b15p66/c1 # budged in giga tones CO2/CO2 emissions per barrel -> budget in billion barrels oil
x2 = x1/c2 # budget in billion barrels / barrels per toe -> budget in Gtoe
budget_mtoe = x2*10e3 # budget in Gtoe * 10^3 -> budget in Mtoe

# emissions per mtoe:
co2_per_toe = c1*c2 # tones of co2 per toe

data['emissions'] = (data[['G']].xs(level='tstep', key=1).max() - data[['G']])*co2_per_toe*10e-4 # emissions in Gtones CO2

In [ ]:
# fraction of resource remaining at success of campaign

# when 2/3 vote for clean investment

data[['Gm']] = data[['G']]#/G_start

data_tmp1 = data.loc[data['decision state']>.66][['emissions']].reset_index()

data_tmp2 = data.xs(level='tstep', key=100)[['emissions']]

data_tmp2

# # for each pair of n_rn, n_cp and sample, select the row with smallest time (the one, where it tipped)
# data_tmp2
data_tmp2['emissions2'] = data_tmp1.loc[data_tmp1.groupby(['phi', 'n_cp', 'sample'])['tstep'].idxmin()].set_index(['phi', 'n_cp', 'sample'])[['emissions']]

for i, row in tqdm_notebook(data_tmp2.iterrows()):
    print(i)
    if np.isfinite(row['emissions2']):
        data_tmp2.loc[i, 'emissions'] = row['emissions2']

In [ ]:

# for that time, look at remaining resource and take the mean over it:
res1_mean = data_tmp2[['emissions']].unstack('sample').mean(axis=1).unstack('n_cp')
res1_std = data_tmp2[['emissions']].unstack('sample').std(axis=1).unstack('n_cp')

res1_min = data_tmp2[['emissions']].unstack('sample').min(axis=1).unstack('n_cp')
res1_max = data_tmp2[['emissions']].unstack('sample').max(axis=1).unstack('n_cp')

# and plot the fraction of remaining resource


fig, ax = plt.subplots(ncols=2, figsize=(12,5))

sn.heatmap(res1_mean, ax=ax[0])
sn.heatmap(res1_std, ax=ax[1])

for axis in ax:
    axis.invert_yaxis()
    every_nth = 5
    for n, label in enumerate(axis.yaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    axis.set_xlabel('initial size of campaign in %')
    axis.set_ylabel(r'rewiring rate $\varphi$')

txt = ['mean cumulative emisions at sucess of campaign', 'standard deviation']
fig.get_axes()[3].set_ylabel(txt[1])
fig.get_axes()[2].set_ylabel(txt[0])

fig.savefig('campaign_vs_phi.pdf')

In [ ]:
# fraction of resource remaining in 2050

t = 40

data_tmp1 = data[['Gm']].xs(level='tstep', key=t).unstack('sample')

res2_mean = data_tmp1.mean(axis=1).unstack('n_cp')
res2_min = data_tmp1.min(axis=1).unstack('n_cp')
res2_max = data_tmp1.max(axis=1).unstack('n_cp')
res2_std = data_tmp1.std(axis=1).unstack('n_cp')

fig, ax = plt.subplots(ncols=4, figsize=(25,5))

sn.heatmap(res2_mean, ax=ax[0])
sn.heatmap(res2_min, ax=ax[1])
sn.heatmap(res2_max, ax=ax[2])
sn.heatmap(res2_std, ax=ax[3])

for axis in ax:
    every_nth = 5
    for n, label in enumerate(axis.yaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)

for ax, text in zip(ax, ['', 'min', 'max', 'std of']):
    ax.invert_yaxis()
    txt = ax.set_title(f'{text} resource remaining in {t+2010}')

In [ ]:
# probability of staying within the 1.5C, p=66% budget at the sucess of the campaign

target = b15p50

# when 2/3 vote for clean investment

data_tmp1 = data.loc[data['decision state']>.66][['emissions']].reset_index()

data_tmp2 = data.xs(level='tstep', key=100)[['emissions']]

# for each pair of n_rn, n_cp and sample, select the row with smallest time (the one, where it tipped)
data_tmp2.loc[data_tmp1.groupby(['phi', 'n_cp', 'sample'])['tstep'].idxmin()] = data_tmp1.loc[data_tmp1.groupby(['phi', 'n_cp', 'sample'])['tstep'].idxmin()]

# for that time, look at remaining resource and take the mean over it:
res1_mean = (data_tmp2.set_index(['phi', 'n_cp', 'sample'])[['emissions']]<target).unstack('sample').mean(axis=1).unstack('n_cp')
res1_std = (data_tmp2.set_index(['phi', 'n_cp', 'sample'])[['emissions']]<target).unstack('sample').sem(axis=1).unstack('n_cp')

In [ ]:
# and plot the fraction of remaining resource

fig1, ax1 = plt.subplots(ncols=2, figsize=(10,4))

sn.heatmap(res1_mean, ax=ax1[0])
sn.heatmap(res1_std, ax=ax1[1])

ax1[0].set_title('probability of keeping the 1.5 budget when 2/3 vote clean')
ax1[1].set_title('standard error of the mean')
for axis in ax1:
    axis.invert_yaxis()
    axis.set_xlabel('initial percentage of campaigners')
    axis.set_ylabel('initial percentage of rednecks')
    axis.tick_params(axis='both', rotation=0)
    #axis.locator_params(axis='both', nbins=5)
    every_nth = 5
    for n, label in enumerate(axis.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    every_nth = 4
    for n, label in enumerate(axis.yaxis.get_ticklabels()):
        if (n - 1) % every_nth != 0:
            label.set_visible(False)
fig1.tight_layout()
fig1.savefig('p_budget15p66_at_campaign_success.pdf')

In [ ]:
# probability of staying within the 1.5C, p=66% budget at year 2010 + t_max

# when 2/3 vote for clean investment

t_max = 40
target = b15p50

data_tmp1 = data.xs(level='tstep', key=t_max)[['emissions']].reset_index()

# for that time, look at remaining resource and take the mean over it:
res2_mean = (data_tmp1.set_index(['phi', 'n_cp', 'sample'])[['emissions']]<target).unstack('sample').mean(axis=1).unstack('n_cp')
res2_std = (data_tmp1.set_index(['phi', 'n_cp', 'sample'])[['emissions']]<target).unstack('sample').sem(axis=1).unstack('n_cp')

In [ ]:
# and plot the fraction of remaining resource

fig2, ax2 = plt.subplots(ncols=2, figsize=(10,4))

sn.heatmap(res2_mean, ax=ax2[0])
sn.heatmap(res2_std, ax=ax2[1])

ax2[0].set_title(f'probability of keeping the 1.5 budget in {2010+t_max}')
ax2[1].set_title('standard error of the mean')
for axis in ax2:
    axis.invert_yaxis()
    axis.set_xlabel('initial percentage of campaigners')
    axis.set_ylabel('initial percentage of rednecks')
    axis.tick_params(axis='both', rotation=0)
    #axis.locator_params(axis='both', nbins=5)
    every_nth = 5
    for n, label in enumerate(axis.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    every_nth = 4
    for n, label in enumerate(axis.yaxis.get_ticklabels()):
        if (n - 1) % every_nth != 0:
            label.set_visible(False)
fig2.tight_layout()
fig2.savefig('p_budget15p50_in2050.pdf')

In [ ]:
fig, ax = plt.subplots()

rednecks = [0, 0.5, 0.8]
colors = ['green', 'orange', 'red']

for c, rn in zip(colors, rednecks):

    df = res2_mean.stack('n_cp').xs(level='phi', key=rn)
    df_s = 2*res2_std.stack('n_cp').xs(level='phi', key=rn)
    x = df.index.values
    y = df.values
    dy = df_s.values

    ax.plot(x, y, '.', color=c)
    ax.fill_between(x, y+dy, y-dy, alpha=.3, color=c)

ax.set_ylim([0,1])

In [ ]:
# Time until emission targets is reached.

# when 2/3 vote for clean investment

data[['Gm']] = data[['G']]

data_tmp1 = data.loc[data['emissions']>b15p50].reset_index()

data_tmp2 = data_tmp1.loc[data_tmp1.groupby(['phi', 'n_cp', 'sample'])['tstep'].idxmin()].set_index(['phi', 'n_cp', 'sample'])[['tstep']]

res3_mean = data_tmp2.unstack('sample').mean(axis=1).unstack('n_cp')
res3_std = data_tmp2.unstack('sample').std(axis=1).unstack('n_cp')
res3_mean

In [ ]:
fig3, ax3 = plt.subplots(ncols=2, figsize=(10,4))

sn.heatmap(res3_mean, ax=ax3[0])
sn.heatmap(res3_std, ax=ax3[1])

for axis, text in zip(ax3, ['fraction of', 'min fraction of', 'max fraction of', 'std of']):
    axis.invert_yaxis()
    axis.set_xlabel('initial percentage of campaigners')
    axis.set_ylabel('initial percentage of rednecks')
    axis.tick_params(axis='both', rotation=0)
    #axis.locator_params(axis='both', nbins=5)
    every_nth = 5
    for n, label in enumerate(axis.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    every_nth = 4
    for n, label in enumerate(axis.yaxis.get_ticklabels()):
        if (n - 1) % every_nth != 0:
            label.set_visible(False)
txt = ax3[0].set_title('time until $1.5^{\circ}$C, $p=0.5$ emission target is reached.')
txt = ax3[1].set_title(f'standard deviation')
fig3.tight_layout()
fig3.savefig('time_until_emissions_target.pdf')

In [ ]:
fig, ax = plt.subplots()

campaign = [0, 10, 15]
colors = ['green', 'orange', 'red']

for c, rn in zip(colors, campaign):

    df = res3_mean.stack('n_cp').xs(level='n_cp', key=rn)
    df_s = res3_std.stack('n_cp').xs(level='n_cp', key=rn)
    x = df.index.values
    y = df.values
    dy = df_s.values

    ax.plot(x, y, '.', color=c)
    ax.fill_between(x, y+dy, y-dy, alpha=.3, color=c)


In [ ]:
res3_mean

In [ ]:
with pd.HDFStore(path + 'X2.h5') as store:
    data = store.select('dat_1', where='phi=0.9 & n_cp = 5', columns=['decision state', 'i_c', 'G'])
data.index = data.index.droplevel(['phi', 'test'])
data.reset_index().drop_duplicates(subset=['sample', 'n_cp']).set_index(['sample', 'n_cp'])
data

In [ ]:
tmp = data.xs(level='tstep', key=40)
tmp['emissions'] = (data.xs(level='tstep', key=1)[['G']].max() - tmp[['G']])*co2_per_toe*10e-4

tmp1 = data.loc[(data['decision state']>.66)].reset_index()

idx = tmp1.groupby(['n_cp', 'sample'])['tstep'].idxmin()

tmp_tip = tmp1.loc[idx].set_index(['n_cp', 'sample'])

tmp['tipping time'] = tmp_tip['tstep']

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))

t = 10
l = [f'tipping before\n{2010+t}', f'tipping in or\nafter {2010+t}']

tmp[l[0]] = float('nan')
tmp[l[1]] = float('nan')

tmp.loc[tmp['tipping time']<t, l[0]] = tmp.loc[tmp['tipping time']<t, 'emissions']
tmp.loc[tmp['tipping time']>=t, l[1]] = tmp.loc[tmp['tipping time']>=t, 'emissions']

l2 = [f'tipping in or\nafter {2010+t}', f'tipping before\n{2010+t}']

tmp[l2].plot(kind='hist',
            ax=ax,
            color=['orange', '#878685'],
            stacked=True,
            bins=20)

ax.axvline(b15p50, label='T2 emission\nbudget')

ax.set_xlabel('cumulative emissions in 2050 [Gton CO2]')
ax.legend()

fig.tight_layout()

fig.savefig('emissions_with_campaign.pdf')

In [ ]:
tmp

In [ ]:
data = data.reset_index().drop_duplicates(subset=['sample', 'n_cp', 'n_rn', 'tstep']).set_index(['sample', 'n_cp', 'tstep'])

data['emissions'] = (data[['G']].xs(level='tstep', key=1).max() - data[['G']])*co2_per_toe*10e-4

data_tmp1 = data.loc[data['emissions']>b15p50].reset_index()

data_tmp2 = data_tmp1.loc[data_tmp1.groupby(['n_rn', 'n_cp', 'sample'])['tstep'].idxmin()].set_index(['n_rn', 'n_cp', 'sample'])[['tstep']]

tmp3 = data_tmp2.unstack('n_rn')
tmp3.columns = tmp3.columns.droplevel(0)
tmp3 = tmp3.reset_index().fillna(100)
len(data_tmp1)

In [ ]:
data_tip1 = data.loc[data['decision state']>.9].reset_index()

data_tip2 = data_tip1.loc[data_tip1.groupby(['n_rn', 'n_cp', 'sample'])['tstep'].idxmin()].set_index(['n_rn', 'n_cp', 'sample'])[['tstep']]

tmp4 = data_tip2.unstack('n_rn')
tmp4.columns = tmp4.columns.droplevel(0)
tmp4 = tmp4.reset_index().fillna(100)
len(data_tip1)

In [ ]:

fig, ax = plt.subplots(figsize=(13, 13))

for rn, color in zip(rednecks, ['r', 'g', 'b']):
    y = tmp4[[rn]]
    x = tmp3[[rn]]
    
    ax.scatter(x=x, y=y,
               alpha=0.8, 
               label=f'initial fraction of rednecks is {rn}%')
ax.legend()
ax.set_xlabel('time when T2 exceeded')
ax.set_ylabel('time when 90% invest clean')
ax.set_xlim([20, 60])

In [ ]:
len(y)